In [2]:
from django.db.backends.dummy.base import ignore
%load_ext autoreload
%autoreload 2

In [3]:
from rixeval import load_chats
import pandas as pd

chats = load_chats.load_chats("/home/finn/Fraunhofer/studydata/db.sqlite3", usernames_to_remove=["finnadmin", "Lena", "test1", "001"])
chats = chats[chats["tracker_length"]!= 1]
answers = load_chats.load_answers("/home/finn/Fraunhofer/studydata/ashri.txt")

data = load_chats.merge(chats, answers)

Found conversation table: dashboard_conversation
Found user table: account_managment_user


/home/finn/Fraunhofer/rixeval/rixeval/load_chats.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_answers['parsed'] = sub_answers['answers'].apply(ast.literal_eval)
/home/finn/Fraunhofer/rixeval/rixeval/load_chats.py:188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_answers[q] = sub_answers['parsed'].apply(lambda x: next((item['answer'] for item in x if item['id'] == q), None))
/home/finn/Fraunhofer/rixeval/rixeval/load_chats.py:188: SettingWithCopyWarning: 
A value is trying to be set on a co

In [4]:
merged_dfs = [x['merged'] for x in data]
df = pd.concat(merged_dfs, ignore_index=True)

In [5]:
from pyalm.models.openai import OpenAI
import os

llm = OpenAI("gpt-4o-2024-08-06", os.getenv("OPENAI_API_KEY"), azure_endpoint=os.getenv("AZURE_ENDPOINT"), n_ctx = 15000)

In [15]:
from rixeval.runner import *

from rixeval import prompt_pieces
from rixeval import prompt_pieces_chats

max_elems = 8
prompt_types = {key: value[:max_elems] for key, value in prompt_pieces_chats.prompts.items()}
direct_answers = prompt_pieces.direct_answers.direct_answers[:max_elems]
cot = prompt_pieces.cot.cot[:max_elems]


variations = {
    "final_prompt_type": NamedCategories("final_prompt_type", prompt_types),
    "think_mode": NamedCategories("think_mode", {"direct": direct_answers, "cot": cot}),
}


from rixeval import dataload
xaidata = dataload.XAIData()

config = ExperimentConfig("temp", "prompt and confidence", variations, verbose=False, no_llm=False,
                          static_parameters= {"include_datapoint_info":"Yes",},)

# #{"confidence":None}
# #{'xai_exclusion': {'shap': True,'pdp': True,'lime_explanation': True,'counterfactual1': True, 'histograms': True}}
ChatExperimentRunner.get_runstats(config, df[:30])
# runner = ChatExperimentRunner(config, df[:2], xaidata, llm, ignore_existing=True)
# results_df = runner.run()
# results_df

Total combinations: 768, Total chats: 30
Total expected runs: 23040
This will take between 691-1114 minutes / 11.52-18.56 hours.


In [5]:
# outcomes = pd.read_csv("../data/explanations_table.csv")
# outcomes.drop(columns=['deposit_type', 'lead_time', 'country',
#        'total_of_special_requests', 'adr', 'arrival_date_week_number',
#        'market_segment', 'arrival_date_day_of_month', 'previous_cancellations',
#        'arrival_date_month', 'stays_in_week_nights', 'booking_changes',
#        'stays_in_weekend_nights', 'reserved_room_type', 'adults', 'hotel',
#        'children', 'lime_explanation', 'shap_explanation'], inplace=True)
# merged_df = df.merge(outcomes,
#                 left_on='current_datapoint_id',
#                       right_on='datapoint_id',
#                       how='left')
# merged_df.drop(columns=['timestamp_decision',"model_name", "current_datapoint_index", "username_decision"], axis=1, inplace=True)
# merged_df